In [1]:
import sys
import pandas as pd
 
sys.path.append('../')

from functions import *

data = pd.read_csv("../../data/demeaned.csv")
df = data.groupby(["id", "ispolice", "sellerfeedbackscore", "bidcount", "apple", "amazon", "increment_residual"])["residual"].apply(lambda x: x.values).reset_index()

/Users/liumukun/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
valid_bids = list(df[df.ispolice == 1].bidcount.value_counts().index)
include = df[(df.bidcount > 1) & (df.bidcount.isin(valid_bids))]

bids = list(include.residual)

logged_feedback = np.log(include.sellerfeedbackscore+1)
logged_feedback = transform_covariates(logged_feedback, 100)
include.sellerfeedbackscore = logged_feedback

covariates = np.array(include[["ispolice", "sellerfeedbackscore"]])
covariates = list([list(cov) for cov in covariates])
incremented = list(include.increment_residual)

<ipython-input-3-72b2804822f4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  include.sellerfeedbackscore = logged_feedback


In [ ]:
median_upper, median_lower = estimate_median(bids, covariates, incremented, (-4,6))
dump(median_lower, median_upper, "CMF_main")

calculating values for covariate: [1.0, 10.094912597722205] (1/99)
total time elapsed: 0.0001593980000000883s
calculating values for covariate: [0.0, 2.05067072014674] (2/99)
total time elapsed: 121.867780628s
calculating values for covariate: [0.0, 6.410962598537746] (3/99)
total time elapsed: 242.443792789s
calculating values for covariate: [0.0, 5.016042433119295] (4/99)
total time elapsed: 383.03907105499997s
calculating values for covariate: [0.0, 4.849364685574428] (5/99)
total time elapsed: 518.99732824s
calculating values for covariate: [0.0, 5.270393880865785] (6/99)
total time elapsed: 688.041211835s
calculating values for covariate: [0.0, 10.59365449453866] (7/99)
total time elapsed: 838.332478059s
calculating values for covariate: [0.0, 6.655428694493396] (8/99)
total time elapsed: 995.444900899s
calculating values for covariate: [0.0, 7.677894253006165] (9/99)
total time elapsed: 1100.123318155s
calculating values for covariate: [0.0, 5.421126171290776] (10/99)
total time 

In [ ]:
results = bootstrap(bids, covariates, incremented, [], [], 40, "CMF_main")